# 离群值检测实验报告：Nair检测法和Grubbs检测法

### 盛焕新      15220202202189

## 1. 离群值检测函数

In [203]:
def find_outliers(df, cv_table, sigma, alpha_jianchu, alpha_tichu, side='both'):
    """
    函数说明：
    df: 用于检测离群值的数据集
    cv_table: 用于表示Nair或Grubbs的临界值表
    sigma: 表示正态分布的标准差
    alpha_jianchu: 表示检出水平
    alpha_tichu: 表示剔除水平
    side: 有三种情况，'upper', 'lower', 'both', 分别代表上侧、下侧和双侧, 默认为双侧
    """
    #首先将数据进行排序，并计算个数及其均值
    data = sorted(df.values.flatten())
    n = len(data)
    u = np.mean(data)
    
    #找出参数情况下的检出水平和剔除水平
    P_jianchu = cv_table.loc[cv_table['n']==n, (1 - alpha_jianchu)].item()
    P_tichu = cv_table.loc[cv_table['n']==n, (1 - alpha_tichu)].item()
    
    #设置两个变量用来统计离群值和歧离值的个数
    i = 0 #表示离群值的个数，第i个
    j = 0 #表示歧离值的个数，第j个

            
     #上侧情形：
    if side == 'upper':
        while True:
            P_last = (data[-1] - u) / sigma
            
            if P_last > P_jianchu: #确认最上侧的值是否高于检出水平
                if P_last > P_tichu:#既然能被检出，确定是否需要剔除
                    i = i+1
                    print(f'检测出第{i}个统计离群值{data[-1]}')#找到需要剔除的值，即为统计离群值
                else:
                    j = j+1
                    print(f'检测出第{j}个歧离值：{data[-1]}')#并没有到需要剔除的水平，即为歧离值
                data = data[: -1]
                continue

            else:   
                print(f'*总结：共检测出{i}个统计离群值和{j}个歧离值！')
                return pd.DataFrame(data, columns=df.columns)

    
    #下侧情形：
    if side == 'lower':
        while True:
            P_first = (u - data[0]) / sigma

            if P_first > P_jianchu:#确认最下侧的值是否高于检出水平
                if P_first > P_tichu: #既然能被检出，确定是否需要剔除
                    i = i+1
                    print(f'检测出第{i}个统计离群值：{data[0]}')#找到需要剔除的值，即为统计离群值
                else:
                    j = j+1
                    print(f'检测出第{j}个歧离值：{data[0]}')#并没有到需要剔除的水平，即为歧离值
                data = data[1: ]
                continue

            else:
                print(f'*总结：共检测出{i}个统计离群值和{j}个歧离值！')
                return pd.DataFrame(data, columns=df.columns)
        
            
    #双侧情形：
    if side == 'both':
        while True:
            P_first = (u - data[0]) / sigma
            P_last = (data[-1] - u) / sigma

            #双侧情况比较复杂，需要比较P_first和P_last的关系
            if P_first == P_last and P_first > P_critical_jianchu:
                    if P_first > P_tichu:
                        i = i+1
                        print(f'检测出第{i}个统计离群值（位于下侧）：{data[0]}')#找到需要剔除的值，即为统计离群值
                    else:
                        j = j+1
                        print(f'检测出第{j}个歧离值（位于下侧）：{data[0]}')#并没有到需要剔除的水平，即为歧离值
                        
                    if P_last > P_tichu:
                        i = i+1
                        print(f'检测出第{i}个统计离群值（位于上侧）：{data[-1]}')#找到需要剔除的值，即为统计离群值
                    else:
                        j = j+1
                        print(f'检测出第{j}歧离值（位于上侧）：{data[-1]}')#并没有到需要剔除的水平，即为歧离值
                    data = data[1: -1]
                    
            elif P_first > P_last and P_first > P_jianchu:
                if P_first > P_tichu:
                    i = i+1
                    print(f'检测出第{i}个统计离群值（位于下侧）：{data[0]}')#找到需要剔除的值，即为统计离群值
                else:
                    j = j+1
                    print(f'检测出第{j}个歧离值（位于下侧）：{data[0]}')#并没有到需要剔除的水平，即为歧离值
                data = data[1: ]        
                    
            elif P_last > P_first and P_last > P_jianchu:
                if P_last > P_tichu:
                    i = i+1
                    print(f'检测出第{i}个统计离群值（位于上侧）：{data[-1]}')#找到需要剔除的值，即为统计离群值
                else:
                    j = j+1
                    print(f'检测出第{j}个歧离值（位于上侧）：{data[-1]}')#并没有到需要剔除的水平，即为歧离值
                data = data[: -1]

            else:   
                print(f'*总结：共检测出{i}个统计离群值和{j}个歧离值！')
                return pd.DataFrame(data, columns=df.columns)

## 2. 实验执行及结果

### 2.1 Nair检测法

In [204]:
import pandas as pd 
import numpy as np

data = pd.read_csv('./Data source/Data source.csv')
cv_Nair = pd.read_excel('./Data source/Critical value table of Nair.xlsx')
cv_Nair.columns = ['n'] + cv_Nair.columns[1: ].map(lambda x: float(x[6: ])).tolist()

print('Nair检测实验结果:')

print('\n①双侧情形:')
Nair_both = find_outliers(df=data, cv_table=cv_Nair, sigma=0.65, alpha_jianchu=0.05, alpha_tichu=0.01)

print('\n②上侧情形:')
Nair_upper = find_outliers(df=data, cv_table=cv_Nair, sigma=0.65, alpha_jianchu=0.05, alpha_tichu=0.01, side='upper')

print('\n③下侧情形:')
Nair_lower = find_outliers(df=data, cv_table=cv_Nair, sigma=0.65, alpha_jianchu=0.05, alpha_tichu=0.01, side='lower')

    """
    调用说明：
    df: 用于检测离群值的数据集
    cv_table: 用于表示Nair或Grubbs的临界值表
    sigma: 表示正态分布的标准差
    alpha_jianchu: 表示检出水平
    alpha_tichu: 表示剔除水平
    side: 有三种情况，'upper', 'lower', 'both', 分别代表上侧、下侧和双侧, 默认为双侧
    """

Nair检测实验结果:

①双侧情形:
检测出第1个统计离群值（位于下侧）：3.13
*总结：共检测出1个统计离群值和0个歧离值！

②上侧情形:
*总结：共检测出0个统计离群值和0个歧离值！

③下侧情形:
检测出第1个统计离群值：3.13
*总结：共检测出1个统计离群值和0个歧离值！


### 2.2 Grubbs检测法

In [205]:
import numpy as np
import pandas as pd 

data = pd.read_csv('./Data source/Data source.csv')
cv_Grubbs = pd.read_excel('./Data source/Critical value table of Grubbs.xlsx')
cv_Grubbs.columns = ['n'] + cv_Grubbs.columns[1: ].map(lambda x: float(x[6: ])).tolist()

print('Grubbs检测实验结果:')

print('\n①双侧情形：')
Grubbs_both = find_outliers(df=data, cv_table=cv_Grubbs, sigma=0.65, alpha_jianchu=0.05, alpha_tichu=0.01)

print('\n②上侧情形:')
Grubbs_upper = find_outliers(df=data, cv_table=cv_Grubbs, sigma=0.65, alpha_jianchu=0.05, alpha_tichu=0.01, side='upper')

print('\n③下侧情形:')
Grubbs_lower = find_outliers(df=data, cv_table=cv_Grubbs, sigma=0.65, alpha_jianchu=0.05, alpha_tichu=0.01, side='lower')


    """
    调用说明：
    df: 用于检测离群值的数据集
    cv_table: 用于表示Nair或Grubbs的临界值表
    sigma: 表示正态分布的标准差
    alpha_jianchu: 表示检出水平
    alpha_tichu: 表示剔除水平
    side: 有三种情况，'upper', 'lower', 'both', 分别代表上侧、下侧和双侧, 默认为双侧
    """

Grubbs检测实验结果:

①双侧情形：
检测出第1个统计离群值（位于下侧）：3.13
检测出第1个歧离值（位于下侧）：3.49
*总结：共检测出1个统计离群值和1个歧离值！

②上侧情形:
*总结：共检测出0个统计离群值和0个歧离值！

③下侧情形:
检测出第1个统计离群值：3.13
检测出第1个歧离值：3.49
*总结：共检测出1个统计离群值和1个歧离值！


## 3. 实验总结：

综上可得，Nair检测法和Grubbs检测法都是检测离群值和歧离值的方法。根据国标数据，可以得出结论：

###   3.1 使用Nair检测法得出结果：

①双侧情形:

    检测出第1个统计离群值（位于下侧）：3.13

*总结：共检测出1个统计离群值和0个歧离值！


②上侧情形:

*总结：共检测出0个统计离群值和0个歧离值！


③下侧情形:

    检测出第1个统计离群值：3.13

*总结：共检测出1个统计离群值和0个歧离值！
   

###   3.2 使用Grubbs检测法得出结果：

①双侧情形：

    检测出第1个统计离群值（位于下侧）：3.13

    检测出第1个歧离值（位于下侧）：3.49

*总结：共检测出1个统计离群值和1个歧离值！

②上侧情形:

*总结：共检测出0个统计离群值和0个歧离值！

③下侧情形:

    检测出第1个统计离群值：3.13

    检测出第1个歧离值：3.49

*总结：共检测出1个统计离群值和1个歧离值！